## Final Project Submission

Please fill out:
* Student name: ANTONY NJOROGE
* Student pace: Part time
* Scheduled project review date/time: 29th April 2025 
* Instructor name: Sam G. Mwangi
* Blog post URL:


# Business Understanding
The company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft. 

Risk in aviation could relate to safety, maintenance costs, operational efficiency, regulatory compliance, or even market demand for certain types of aircraft.

## Objective: 
Determining which aircraft are the lowest risk for the company to start this new business endeavor.
Translate my findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.

## Data Understanding
The dataset ['AviationData.csv'](https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses) is about accidents, not operational data. It contains data about Civil aviation accidents and selected incidents within the United States, its territories and possessions, and in international waters. 

### Methodology
1. Filter data to relevant columns

2. Dealing with missing values

3. Find the top 2 models with least fatality rates for both commercial and     private purposes.

4. Visualization 


## Setup

Import the relevant libraries, open up the **AviationData.csv** file as dataframe called **df**. 




In [8]:
# Import pandas and matplotlib
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file into a DataFrame
df = pd.read_csv('data\AviationData.csv', encoding='latin1')

# Display the various datatypes within the dataframe
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

C:\Users\VIDIC\AppData\Local\Temp\ipykernel_10024\2049371704.py:6: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data\AviationData.csv', encoding='latin1')


## 1. Filter data to relevant columns

In [9]:
# Drop columns with more than 50% missing values
columns_to_drop_1 = ['Latitude', 'Longitude', 'Aircraft.Category', 'FAR.Description', 'Schedule', 'Air.carrier','Airport.Code', 'Airport.Name']
df = df.drop(columns=columns_to_drop_1)
df.isnull().sum()
df.shape

(88889, 23)

## 2. Data cleaning

In [10]:
# Drop rows with missing values in 'Make' , 'Model' and 'Location columns
df.dropna(subset=['Make', 'Model', 'Location'], inplace=True)
df.shape

(88725, 23)

In [11]:
# Strip whitespaces
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)